In [1]:
import torch
import matplotlib.pyplot as plt
from torch import optim, distributions, nn
import torch.nn.utils as nn_utils
from tqdm import tqdm
from gpzoo.gp import SVGP, VNNGP
from gpzoo.kernels import NSF_RBF, RBF
from gpzoo.likelihoods import NSF2
from gpzoo.utilities import rescale_spatial_coords, dims_autocorr, regularized_nmf, add_jitter, scanpy_sizefactors #init_softplus
import gpzoo.pri_experiment_utilities as exp
import squidpy as sq
import numpy as np
import time
import random
import scanpy as sc
import anndata as ad
from anndata import AnnData
from squidpy.gr import spatial_neighbors, spatial_autocorr
from sklearn.cluster import KMeans
from os import path

/engelhardt/home/pshrestha/vnngp/GPzoo/gpzoo/utilities.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
/engelhardt/home/pshrestha/anaconda3/envs/vnngp2/lib/python3.9/site-packages/dask/dataframe/_pyarrow_compat.py:15: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 13.0.0. Please consider upgrading.
  warnings.warn(


In [2]:
random.seed(256)
root_path = '/engelhardt/home/pshrestha/vnngp/'

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [4]:
L = [5]
#L = [15]
#N = [3000]
M = [100]
K = [1]
lkzz_builds = [3]

In [5]:
save_path = path.join(root_path, "results/visium/nnnsf")

#outputs_dict = defaultdict(list)
for l in L:
    for m in M:
        for k in K:
            for lk in lkzz_builds:
                file_path = path.join(save_path, f"VNNGP_K={k}_lkzz={lk}_M={m}_L={l}_lr=0.001_jtr=0.01_iter=6000_bs=128_NMFinit_state_dict.pth")
                if not path.isfile(file_path):
                    print(f"{file_path} does not exist. Running the code...")
                    dicts = {
                            'L': l, 
                            'M': m, 
                            'K': k,
                            'sigma': 0.1, 
                            'lengthscale': 0.07, 
                            'jtr': 1e-2,
                            'batch_size': 128,
                            'lr': 1e-3,
                            'rs': 256,
                            'lkzz_build': lk,
                    }
                    nmf_path = path.join(root_path, f"results/visium/nmf/nmf_factors_iter=1000_rs=256_L={dicts['L']}.pt")
                    exp.run_experiment(exp.load_visium, save_path=save_path, steps=6000, batched=True,
                                     model_type='VNNGP', NMF=True, nmf_path=nmf_path, kwargs=dicts)
                else:
                    print(f"{file_path} exists. Skipping the code.")

#def run_experiment(data_func, save_path, steps=1000, batched=False, model_type=None, NMF=False, nmf_path=None, **kwargs):
#nmf_path = f"./results/nmf/nmf_factors_iter=1000_rs=256_L={dicts['L']}.pt"
#nmf_path = f"./visium_final/nmf/nmf_factors_iter=500_rs=256_L={dicts['L']}.pt"

/engelhardt/home/pshrestha/vnngp/results/visium/nnnsf/VNNGP_K=1_lkzz=3_M=100_L=5_lr=0.001_jtr=0.01_iter=6000_bs=128_NMFinit_state_dict.pth does not exist. Running the code...


  0%|                                                               | 0/6000 [00:00<?, ?it/s]


lkzz == 3


RuntimeError: expand(torch.cuda.FloatTensor{[128, 128, 1, 100]}, size=[128, 1, 1]): the number of sizes provided (3) must be greater or equal to the number of dimensions in the tensor (4)